In [28]:
# Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets 
import pandas as pd
import time
import requests
from census import Census
pd.set_option('display.max_rows', 30000)
# Census API Key
from config import census_key
c = Census(census_key, year=2015)

### Retrieve Crime Data from FBI Crime Database - UCR

In [18]:
#Store file path for crime data in a variable
crime_data = "../Data/Offenses_Known_to_Law_Enforcement_by_State_by_City_2015.xls"

#Read data into dataframe
crime_data_df=pd.read_excel(crime_data)

#Extract the state into a list so all rows in "state" column can have a state value
state = crime_data_df["State"]
states=[]
for s in state:
    if pd.isnull(s):
        states.append(current_state)
    else:
        current_state = s
        states.append(current_state)
        
#Add newly created list into the dataframe and remove the old column
crime_data_df["NEW_STATE"] = states
clean_crime_data_df=crime_data_df[["NEW_STATE", "City", "Population", "Violent Crime"]] 
clean_crime_data_df=clean_crime_data_df.rename(columns={"NEW_STATE":"State"})

#Remove any trailing numbers at the end of state and city names
clean_crime_data_df["State"] = clean_crime_data_df["State"].map(lambda x: x.rstrip("1234567890"))
clean_crime_data_df["City"] = clean_crime_data_df["City"].map(lambda x: x.rstrip("1234567890"))

#Calculate the rate of violent crime per capita and remove any city where the violent crime is 0
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]
clean_crime_data_df["Violent Crime Rate"] = (clean_crime_data_df["Violent Crime"]/clean_crime_data_df["Population"])*100

clean_crime_data_df=clean_crime_data_df.sort_values("State")
clean_crime_data_df = clean_crime_data_df.loc[clean_crime_data_df["Violent Crime"]>0]

In [29]:
clean_crime_data_df

,State,City,Population,Violent Crime,Violent Crime Rate
0,ALABAMA,Abbeville,2610,9.0,0.344828
122,ALABAMA,Moulton,3353,13.0,0.387712
123,ALABAMA,Moundville,2467,6.0,0.243210
124,ALABAMA,Mountain Brook,20801,14.0,0.067304
125,ALABAMA,Mount Vernon,1557,12.0,0.770713
126,ALABAMA,Muscle Shoals,13720,45.0,0.327988
127,ALABAMA,New Brockton,1152,1.0,0.086806
128,ALABAMA,New Hope,2815,3.0,0.106572
129,ALABAMA,Newton,1463,4.0,0.273411
130,ALABAMA,Northport,25032,97.0,0.387504


## Sort Out the Highest and Lowest Crime Rate for Each State

In [25]:
#Sort out the highest crime rate for each state
#variable to hold highest violent crime for each state
violent_crime=[]

#variable to track highest violent crime for each state
violent_crime_track=0

#variable to hold starting state for the loop
current_state='ALABAMA'

#variable to append city for highest crime rate
city=[]

#variable to append city for highest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if row['State'] == current_state:
        if row["Violent Crime Rate"]> violent_crime_track:
            highest = row["Violent Crime Rate"] 
            cur_city = row["City"]
            cur_state = row["State"]
            violent_crime_track = highest
               
    else:
        violent_crime.append(highest)
        highest = row["Violent Crime Rate"]
        violent_crime_track = highest
        current_state=row['State']
        city.append(cur_city)
        state.append(cur_state)
        cur_city = row["City"]
        cur_state = row["State"]
#append the data for the last state
violent_crime.append(highest)
city.append(cur_city)
state.append(cur_state)

#Place newly created lists into dictionary and convert to DataFrame
highest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
highest_new_df = pd.DataFrame(highest_new_dict)
highest_new_df.to_csv("../Analysis/highest_crime.csv", index=False)

In [26]:
#Sort out the lowest crime rate for each state
#variable to hold lowest violent crime for each state
violent_crime=[]

#variable to track lowest violent crime for each state
violent_crime_track=100

#variable to hold starting state for the loop
current_state='ALABAMA'

#variable to append city for lowest crime rate
city=[]

#variable to append city for lowest crime rate
state=[]

#Loop through each row in dataframe 
for index,row in clean_crime_data_df.iterrows():
    if row['State'] == current_state:
        if row["Violent Crime Rate"]< violent_crime_track:
            lowest = row["Violent Crime Rate"] 
            cur_city = row["City"]
            cur_state = row["State"]
            violent_crime_track = lowest
               
    else:
        violent_crime.append(lowest)
        lowest = row["Violent Crime Rate"]
        violent_crime_track = lowest
        current_state=row['State']
        city.append(cur_city)
        state.append(cur_state)
        cur_city = row["City"]
        cur_state = row["State"]
        
#append the data for the last state
violent_crime.append(lowest)
city.append(cur_city)
state.append(cur_state)

#Place newly created lists into dictionary and convert to DataFrame
lowest_new_dict = {'State':state,'City':city,'Violent Crime Rate':violent_crime}
lowest_new_df = pd.DataFrame(lowest_new_dict)
lowest_new_df.to_csv("../Analysis/lowest_crime.csv", index=False)